## Make RGB jpeg image from previously stacked and aligned FITS images

### Written and revised (2016-2019) by J. Lowenthal, M. Petersen, K. Ward-Duong 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits
from astropy import stats
from PIL import Image

In [ ]:
# EDIT AS NEEDED

targname =['NGC6853']

filters=['Blue','Visual','Red']  # In order of increasing wavelength: BGR, not RGB

# Provide the path to your data directory, the folder with your *stacked* reduced science images:
# The path format should be the absolute path, e.g., "/Users/StudentName/AST337/MyData/Stacked/" 

# (note there IS a '/' at the end this time!)
datadir = 

siglowhi = [-0.5,20.,-0.5,20.,-0.5,20.] # BGR low and high sigma limits. Ex: To make red brighter, make 6th # lower.

In [ ]:
Rtmp = fits.getdata(datadir+targname[0]+'_'+filters[2]+'stack.fits')
print(np.mean(Rtmp))
print(datadir+targname[0]+'_'+filters[2]+'stack.fits')

In [ ]:
def scale_filter(tmpimg,lowsig=-1.,highsig=15.):

    tmpimg -= np.nanmedian(tmpimg)
    print('minmax 1: ', np.nanmin(tmpimg),np.nanmax(tmpimg))


    tmpsig = stats.sigma_clipped_stats(tmpimg, sigma=2, iters=5)[2]
    print('std: ', tmpsig)
    print("lowsig, highsig: ", lowsig, highsig)
    print('cuts: ', lowsig*tmpsig, highsig*tmpsig)

    image_hist = plt.hist(tmpimg.flatten(), 1000, range=[-100,100])

    # apply thresholding
    tmpimg[np.where(tmpimg < lowsig*tmpsig)] = lowsig*tmpsig
    tmpimg[np.where(tmpimg > highsig*tmpsig)] = highsig*tmpsig
    print('minmax 2: ', np.nanmin(tmpimg),np.nanmax(tmpimg))

    # double hyperbolic arcsin scaling
    tmpimg = np.arcsinh(tmpimg)
    print('minmax 3: ', np.nanmin(tmpimg),np.nanmax(tmpimg))

    # scale to [0,255]
    tmpimg += np.nanmin(tmpimg)
    tmpimg *= 255./np.nanmax(tmpimg)
    tmpimg[np.where(tmpimg < 0.)] = 0.
    print('minmax 4: ', np.nanmin(tmpimg),np.nanmax(tmpimg))
    
    # recast as unsigned integers for jpeg writer
    IMG = Image.fromarray(np.uint8(tmpimg))
    
    print("")
    
    return IMG

# Read in 3 images 
Rtmp = fits.getdata(datadir+targname[0]+'_'+filters[2]+'stack.fits')
Gtmp = fits.getdata(datadir+targname[0]+'_'+filters[1]+'stack.fits')
Btmp = fits.getdata(datadir+targname[0]+'_'+filters[0]+'stack.fits')

# Scale all 3 images
print('Calculating stats....')
R = scale_filter(Rtmp,lowsig=siglowhi[4],highsig=siglowhi[5])
G = scale_filter(Gtmp,lowsig=siglowhi[2],highsig=siglowhi[3])
B = scale_filter(Btmp,lowsig=siglowhi[0],highsig=siglowhi[1])

# Merge 3 images into one RGB image
im = Image.merge("RGB", (R,G,B))

im.save(datadir+targname[0]+'_RGB.jpg', "JPEG")
print("Saved image as ", datadir+targname[0]+'_RGB.jpg')